# Yiddish TTS — Colab setup

This notebook sets up a Colab environment that can run the `generate_yiddish_speech.py` script from your repository.

Follow cells in order. If you use GPU, switch Runtime -> Change runtime type -> GPU. For Coqui TTS compatibility, CPU runtime is acceptable but slower.

In [ ]:
<VSCode.Cell language="markdown">## 1) Mount Google Drive

Mount Drive so you can persist outputs and upload your repository ZIP into Drive.</VSCode.Cell>

<VSCode.Cell language="python">from google.colab import drive
drive.mount('/content/drive')
# Create a working folder in Drive for this notebook
import os
workdir = '/content/drive/MyDrive/yiddish_tts_colab'
os.makedirs(workdir, exist_ok=True)
print('Working directory:', workdir)
print('Drive mounted; you can upload your repo ZIP to this folder and unzip it here.')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 2) Runtime & GPU/TPU (Colab-specific)

Check for available accelerators and instructions to enable GPU. TTS inference benefits from GPU, but CPU works for small tests.</VSCode.Cell>

<VSCode.Cell language="python">import torch
print('Python version:', __import__('sys').version)
print('Torch available:', bool(torch.__version__))
print('CUDA available:', torch.cuda.is_available())
# If CUDA is available, show device name
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="markdown">## 3) Install and Configure Dependencies

Install CPU-friendly PyTorch (or GPU variant if you switched runtime), then Coqui TTS and required audio libs. Run the cell below. This may take several minutes.</VSCode.Cell>

<VSCode.Cell language="python"># Install CPU PyTorch wheel and Coqui TTS
# If you switched to GPU runtime, replace the torch install command with the recommended CUDA command from pytorch.org
!pip install --upgrade pip setuptools wheel
!pip install --quiet --no-cache-dir torch==2.2.2+cpu --index-url https://download.pytorch.org/whl/cpu
!pip install --quiet --no-cache-dir TTS soundfile librosa numpy==1.24.3 pandas scipy matplotlib scikit-learn PyYAML tqdm tensorboard psutil torchvision openai-whisper faster-whisper whisperx==0.5.1
# Whisper packages are optional; adjust if you don't need them
print('Packages installed (or already present).')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 4) Import Libraries & Reproducibility

Import main libraries and set seeds for reproducibility.</VSCode.Cell>

<VSCode.Cell language="python">import random, numpy as np, os
import torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
print('Imports OK; seeds set.')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 5) Upload or Download Repository / Files

Options:
- Upload a ZIP of your repo to the Drive working folder created earlier and unzip it.
- Or use the upload widget below to upload single files (e.g., `generate_yiddish_speech.py` and a reference audio file).
</VSCode.Cell>

<VSCode.Cell language="python"># File upload widget (for small uploads)
from google.colab import files
print('If you have a small repo or just the script, use the widget to upload generate_yiddish_speech.py and any reference audio.')
# uploaded = files.upload()  # uncomment to use interactive upload
print('Or upload a repo ZIP to your Drive at', workdir, 'and unzip it manually with the next cell.')
</VSCode.Cell>

<VSCode.Cell language="python"># Example: unzip a repo ZIP placed in Drive working dir
import os
repo_zip = os.path.join(workdir, 'yiddish_TTS_improved.zip')
if os.path.exists(repo_zip):
    print('Found ZIP, extracting...')
    !unzip -q "$repo_zip" -d /content/yiddish_repo
    print('Extracted to /content/yiddish_repo')
else:
    print('No ZIP found at', repo_zip)
</VSCode.Cell>

In [ ]:
<VSCode.Cell language="markdown">## 6) Data Loading and Preprocessing (reference audio check)

Ensure your dataset reference audio exists or upload a replacement file.</VSCode.Cell>

<VSCode.Cell language="python"># Check for the default reference audio path used by the script
default_ref = '/content/yiddish_repo/tts_segments/audio/segment_0001.wav'
if os.path.exists(default_ref):
    print('Reference audio found:', default_ref)
else:
    print('Reference audio not found at default location.')
    print('If you have a local reference audio, upload it now:')
    # files.upload()  # uncomment to interactively upload a reference audio file
</VSCode.Cell>

<VSCode.Cell language="markdown">## 7) Core functions — run the `generate_yiddish_speech.py` script

This cell runs the repository script with a custom Yiddish phrase and shows output. Adjust paths if you unzipped to a different folder.</VSCode.Cell>

<VSCode.Cell language="python"># Path to the repo script
script_path = '/content/yiddish_repo/generate_yiddish_speech.py'
if not os.path.exists(script_path):
    # Fallback to searching current working dir if you uploaded just the script
    script_path = '/content/generate_yiddish_speech.py'

if not os.path.exists(script_path):
    raise FileNotFoundError('generate_yiddish_speech.py not found in notebook. Upload it or unzip repo into Drive and rerun this cell.')

# Example run with a custom Yiddish phrase
yiddish_text = 'שבת שלום, ווי גייט עס?'
output_file = '/content/yiddish_output.wav'
print('Running script:', script_path)
!python3 "$script_path" "$yiddish_text"
print('If the script ran, output saved to', output_file, 'or look in the repo-specific sample folder.')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 8) Run sample-phrases (no args)

Run the script without arguments to generate the built-in sample phrases (saved into the repo directory).</VSCode.Cell>

<VSCode.Cell language="python">!python3 /content/yiddish_repo/generate_yiddish_speech.py || true
print('Script executed (errors shown above if any). Check yiddish_speech_samples/ in the repo folder for outputs.')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 9) Save outputs and checkpoints to Drive

Copy any generated WAVs or checkpoints back to Drive to persist them.</VSCode.Cell>

<VSCode.Cell language="python"># Example: copy outputs to Drive working folder
!mkdir -p "/content/drive/MyDrive/yiddish_tts_colab/outputs"
!cp -r /content/yiddish_repo/yiddish_speech_samples /content/drive/MyDrive/yiddish_tts_colab/outputs || true
print('Copied any generated samples to Drive: /content/drive/MyDrive/yiddish_tts_colab/outputs')
</VSCode.Cell>

<VSCode.Cell language="markdown">## 10) Basic runtime checks and unit tests

A few asserts to validate the environment and basic functions.</VSCode.Cell>

<VSCode.Cell language="python"># Check that TTS can be imported
try:
    from TTS.api import TTS
    print('TTS import OK')
except Exception as e:
    print('TTS import failed:', e)
# Check torch import and device
import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
</VSCode.Cell>

<VSCode.Cell language="markdown">## 11) Visualize results

You can listen to output files or plot spectrograms. Example to display an audio player for an output file.</VSCode.Cell>

<VSCode.Cell language="python">from IPython.display import Audio, display
example_out = '/content/yiddish_output.wav'
if os.path.exists(example_out):
    display(Audio(example_out))
else:
    print('No example output available at', example_out)
</VSCode.Cell>